<a href="https://colab.research.google.com/github/mhuckvale/pals0039/blob/master/Answers_8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![PALS0039 Logo](https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png)](https://www.phon.ucl.ac.uk/courses/pals0039/)

# Exercise 8.1 Answers

In this exercise we use a seq2seq model to address the problem of grapheme to phoneme conversion. We use an English pronunciation dictionary called BEEP which has been converted to the SAMPA machine readable prounciation format. This exercise draws on the Keras example for seq2seq processing: [https://keras.io/examples/lstm_seq2seq/](https://keras.io/examples/lstm_seq2seq/)

(a) Set up

In [0]:
import numpy as np
import pandas as pd

%tensorflow_version 2.x
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

---
(b) Load the pronunciation dictionary

In [0]:
url="https://www.phon.ucl.ac.uk/courses/pals0039/data/english-dictionary.csv"

df=pd.read_csv(url,keep_default_na=False)
print(df.head())
print(df.tail())

---
(c) Build the input and output vocabulary of symbols. Run the code and add comments.

In [0]:
# get list of all characters used in ortography
inputs=df.INPUT.to_list()
input_chars=sorted(list(set("".join(inputs))))
# add space as a special character
input_chars.append(' ')
print(input_chars)
# get list of all the characters used in transcription
targets=df.TARGET.to_list()
target_chars=sorted(list(set("".join(targets))))
# add space and square brackets as special characters
target_chars.append('[')
target_chars.append(']')
target_chars.append(' ')
print(target_chars)

---
(d) Get basic parameters for learning. Run the code and add comments

In [0]:
# number of pronunciations
num_samples=len(df)
# number of different tokens used in orthography
num_encoder_tokens = len(input_chars)
# number of differet tokens used in transcription
num_decoder_tokens = len(target_chars)
# maximum lengths of words and pronunciations
max_encoder_seq_length = max([len(txt) for txt in df.INPUT.to_list()])
max_decoder_seq_length = max([len(txt) for txt in df.TARGET.to_list()])+2   # plus 2 for BOS=[ and EOS=]

print('Number of samples:', num_samples)
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# build dictionaries for letters and phones
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_chars)])


---
(e) Build training data. Run the code and add comments.

In [0]:
# list of sequences to be used for encoder input
encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens),dtype='float32')
# list of sequences to be used for decoder input (= target shifted right one place)
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens),dtype='float32')
# list of sequences to be used for decoder output
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens),dtype='float32')

# encode orthogaphy as sequence of one-hot vectors
def encode_str(str):
  input_text=str.ljust(max_encoder_seq_length)
  data=np.zeros((max_encoder_seq_length,num_encoder_tokens))
  for t, char in enumerate(input_text):
    data[t, input_token_index[char]] = 1.
  return data

# encode all pronunciations as sequences of one-hot vectors
for i in range(num_samples):
  encoder_input_data[i,:,:]=encode_str(df.INPUT.at[i])
  target_text=("["+df.TARGET.at[i]+"]").ljust(max_decoder_seq_length)
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# random shuffle
nseq=encoder_input_data.shape[0]
p = np.random.permutation(nseq)
encoder_input_data = encoder_input_data[p]
decoder_input_data = decoder_input_data[p]
decoder_target_data = decoder_target_data[p]

print(encoder_input_data.shape,decoder_input_data.shape,decoder_target_data.shape)

---
(f) Build the seq2seq model for training. Run the code and add comments.

In [0]:
# Latent dimensionality of the encoding space.
latent_dim = 256  

# Define the input sequence and process it through LSTMs to get encoder state at end
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

---
(g) Train the seq2seq encoder. Training takes about 5 minutes. Run the code and add comments.

In [0]:
epochs = 5  # Number of epochs to train for.
batch_size = 64  # Batch size for training.

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.05)


---
(h) Build inference model. Run the code and add comments

In [0]:
# Next: inference mode (sampling).
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state and a "start of sequence" token as input.
# Output will be the next target token
# 3) Repeat with this target as input and the current state passed forward

# Define sampling model
encoder_model = Model(encoder_inputs, encoder_states)
# take state from encoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# decoder LSTM takes decoder state and decoder input to produce one new output
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
  # reshape the input into a single row tensor
  input_tensor=np.reshape(input_seq,(1,input_seq.shape[0],input_seq.shape[1]))
  # Encode the input as state vectors.
  states_value = encoder_model.predict(input_tensor)
  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first character of target sequence with the start character.
  target_seq[0, 0, target_token_index['[']] = 1.

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]

    # Exit condition: either hit max length
    # or find stop character.
    if (sampled_char == ']' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
    else:
      decoded_sentence += sampled_char

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # Update states
    states_value = [h, c]

  return decoded_sentence

---
(i) Try out model with some samples. Think of some words to test the pronunciation accuracy.

In [0]:
examples=[ "england", "wales", "scotland", "oxford", "greenhouse"]
for ex in examples:
  data=encode_str(ex.upper())
  decoded=decode_sequence(data)
  print('-')
  print('Input:', ex)
  print('Pronunciation:', decoded)

In [0]:
while True:
  ex=input("Type word (q to quit): ")
  if ex=="q":
    break
  data=encode_str(ex.upper())
  decoded=decode_sequence(data)
  print('Input:', ex)
  print('Pronunciation:', decoded)